<a href="https://colab.research.google.com/github/AdopleAIOrg/Table-Question-Answering/blob/main/Table_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r /content/requirements.txt

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import tempfile
from transformers import AutoModelForSeq2SeqLM, AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline

class TableQuestionAnswering:
    """
    A class that performs table question answering using the TAPAS model.
    """

    def __init__(self):
        """
        Initializes the TableQuestionAnswering class by loading the TAPAS model and tokenizer.
        """

        model_tapas = "google/tapas-large-finetuned-wtq"
        tokenizer_tapas = AutoTokenizer.from_pretrained(model_tapas)
        self.pipe_tapas = pipeline("table-question-answering", model=model_tapas, tokenizer=tokenizer_tapas)

    def _processing_file(self, query: str, file: str, correct_answer: str) -> str:
        """
        Processes the uploaded file and generates an answer for the given query using the TAPAS model.

        Args:
            query (str): The question to ask about the table.
            file (str): The path to the uploaded file.
            correct_answer (str): The correct answer (optional).

        Returns:
            str: The generated answer.
        """

        table = pd.read_csv(file.name).astype(str).fillna('')
        result_tapas = self.pipe_tapas(table=table, query=query)
        return result_tapas['answer']

    def _generate_answer(self, query: str, file: str, correct_answer: str) -> str:
        """
        Generates an answer for the given query and file using the TAPAS model.

        Args:
            query (str): The question to ask about the table.
            file (str): The path to the uploaded file.
            correct_answer (str): The correct answer (optional).

        Returns:
            str: The generated answer.
        """

        excel_file = pd.read_excel(file.name)
        excel_file.to_csv(f"{file.name}.csv",index=False)
        file_name = f"{file.name}.csv"
        table = pd.read_csv(file_name).astype(str).fillna('')
        result_tapas = self.pipe_tapas(table=table, query=query)
        return result_tapas['answer']

    def streamlit_interface(self):
        """
        Creates the Streamlit interface for the Table QA application.
        """

        # Set up the Streamlit app
        st.set_page_config(page_title="Streamlit Table QA")

        st.title("Table QA")
        input_file = st.file_uploader("Upload a CSV file", type=['csv','xlsx'])

        if input_file is not None:
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(input_file.read())
                temp_file.flush()
                query = st.text_input("Enter a question")
                if query != "":
                    st.write("Selected file: ", input_file.name)
                    if input_file.name.endswith('.csv'):
                      result_answer = self._processing_file(query, temp_file, "")
                      st.write("Answer:", result_answer)
                    elif input_file.name.endswith('.xlsx'):
                      result_answer = self._generate_answer(query, temp_file, "")
                      st.write("Answer:", result_answer)
                else:
                    st.write("Please enter a query")

if __name__ == "__main__":

    table_qa = TableQuestionAnswering()
    table_qa.streamlit_interface()

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501